In [1]:
import pickle

with open("data/MicroLens_1M_x1/feature_processor.pkl", "rb") as f:
    feature_processor = pickle.load(f)
print(feature_processor)

In [12]:
import pandas as pd
data_name = "EMB_all_emb_cb08_d"
new_emb = pd.read_parquet(f"data/MicroLens_1M_x1/{data_name}.parquet")
item_feature = pd.read_parquet("dataset/MicroLens_1M_MMCTR/item_feature.parquet")[["item_id", "likes_level",  "views_level"]]

print(item_feature.head())

   item_id  likes_level  views_level
0        1            7            2
1        2            5            9
2        3            2            2
3        4            2            1
4        5            7            5


In [13]:
all_emb = pd.merge(new_emb, item_feature, on="item_id", how="left")

In [14]:
all_emb = all_emb.rename(columns={"likes_level": "likes", "views_level": "views"})
# all_emb.fillna(0, inplace=True)
print(all_emb.head())

   item_id        item_tags  \
0        0  [0, 0, 0, 0, 0]   
1        1  [0, 0, 0, 0, 1]   
2        2  [0, 0, 2, 3, 4]   
3        3  [0, 0, 5, 6, 7]   
4        4  [0, 0, 0, 8, 9]   

                                       item_emb_d128  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [-0.587724506855011, -0.38462838530540466, 0.4...   
2  [-2.5054404735565186, 1.5605803728103638, 0.23...   
3  [-1.0391175746917725, 1.031670093536377, -0.45...   
4  [0.1128326952457428, -0.7956982254981995, 0.70...   

                                             dis_emb  likes  views  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...    NaN    NaN  
1  [225, 52, 899, 895, 752, 91, 610, 907, 633, 22...    7.0    2.0  
2  [35, 662, 419, 476, 639, 357, 92, 907, 425, 81...    5.0    9.0  
3  [887, 52, 789, 109, 710, 1007, 568, 335, 843, ...    2.0    2.0  
4  [35, 662, 966, 526, 353, 908, 624, 720, 630, 3...    2.0    1.0  


In [15]:
all_emb.to_parquet(f"data/MicroLens_1M_x1/{data_name}.parquet")

In [13]:
import pandas as pd
data = pd.read_parquet("dataset/MicroLens_1M_MMCTR/item_seq.parquet")


In [16]:
data.head()

,user_id,item_seq
0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
from tqdm import tqdm
lens = []
for i, item in tqdm(data.iterrows(), total=len(data)):
    one_item = item["item_seq"]
    while(one_item[0] == 0):
        one_item = one_item[1:]
    lens.append(len(one_item))

data["len"] = lens
data["len"].describe()  

100%|██████████| 6000000/6000000 [07:50<00:00, 12746.39it/s]


count    6.000000e+06
mean     7.973784e+00
std      7.234855e+00
min      3.000000e+00
25%      4.000000e+00
50%      6.000000e+00
75%      9.000000e+00
max      1.000000e+02
Name: len, dtype: float64

In [1]:
# add gpt/qwen embedding
import pandas as pd
import numpy as np
raw_data = pd.read_parquet("dataset/MicroLens_1M_MMCTR/item_feature_gpt.parquet")
qwen_data = np.load("dataset/qw_data_all.npy").tolist()
raw_data["img_emb_QWEN"] = qwen_data
raw_data.to_parquet("dataset/MicroLens_1M_MMCTR/item_feature_gpt_qwen.parquet")

In [ ]:
import torch
import pandas as pd
raw_data = pd.read_parquet("dataset/MicroLens_1M_MMCTR/item_feature.parquet")
item_info = pd.read_parquet("data/MicroLens_1M_x1/item_info.parquet")
clip_disc_data = torch.load("data/indices_clip.pt")
gpt_disc_data = torch.load("data/indices_gpt.pt")
# dis_emb = torch.cat([clip_disc_data, gpt_disc_data], dim=1)
# print(dis_emb.shape)
new_data = pd.DataFrame()
new_data["item_id"] = raw_data["item_id"]
new_data["dis_emb_clip"] = clip_disc_data.tolist()
new_data["dis_emb_gpt"] = clip_disc_data.tolist()
new_data = pd.merge(item_info, new_data, on="item_id", how="left")
new_data.at[0, "dis_emb_clip"]=[0 for i in range(len(clip_disc_data[0]))]
new_data.at[0, "dis_emb_gpt"]=[0 for i in range(len(gpt_disc_data[0]))]
print(new_data.head())


/home/yyshao/anaconda3/envs/fuxictr/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


   item_id        item_tags  \
0        0  [0, 0, 0, 0, 0]   
1        1  [0, 0, 0, 0, 1]   
2        2  [0, 0, 2, 3, 4]   
3        3  [0, 0, 5, 6, 7]   
4        4  [0, 0, 0, 8, 9]   

                                       item_emb_d128  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [-0.587724506855011, -0.38462838530540466, 0.4...   
2  [-2.5054404735565186, 1.5605803728103638, 0.23...   
3  [-1.0391175746917725, 1.031670093536377, -0.45...   
4  [0.1128326952457428, -0.7956982254981995, 0.70...   

                                        dis_emb_clip  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [35, 9, 41, 18, 34, 1, 58, 8, 53, 8, 50, 49, 9...   
2  [35, 23, 6, 5, 27, 5, 32, 19, 23, 4, 16, 42, 2...   
3  [21, 9, 41, 18, 34, 55, 6, 37, 43, 4, 10, 4, 4...   
4  [46, 9, 6, 11, 22, 48, 6, 15, 24, 20, 22, 33, ...   

                                         dis_emb_gpt  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1  [35, 9, 41, 18, 34, 1, 58,

In [3]:
new_data.to_parquet("data/MicroLens_1M_x1/item_info_ljh2.parquet")

In [17]:

import pandas as pd
import numpy as np
import yaml 
import os
info_data = pd.read_parquet("data/MicroLens_1M_x1/item_info_ljh2.parquet")
info_data2 = pd.read_parquet("data/MicroLens_1M_x1/EMB_all_emb_cb08_d.parquet")
clip_feature = info_data["dis_emb_clip"].tolist()
info_data2["dis_emb_clip"] = clip_feature 
print(info_data2.head())


   item_id        item_tags  \
0        0  [0, 0, 0, 0, 0]   
1        1  [0, 0, 0, 0, 1]   
2        2  [0, 0, 2, 3, 4]   
3        3  [0, 0, 5, 6, 7]   
4        4  [0, 0, 0, 8, 9]   

                                       item_emb_d128  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [-0.587724506855011, -0.38462838530540466, 0.4...   
2  [-2.5054404735565186, 1.5605803728103638, 0.23...   
3  [-1.0391175746917725, 1.031670093536377, -0.45...   
4  [0.1128326952457428, -0.7956982254981995, 0.70...   

                                             dis_emb  likes  views  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...    NaN    NaN   
1  [225, 52, 899, 895, 752, 91, 610, 907, 633, 22...    7.0    2.0   
2  [35, 662, 419, 476, 639, 357, 92, 907, 425, 81...    5.0    9.0   
3  [887, 52, 789, 109, 710, 1007, 568, 335, 843, ...    2.0    2.0   
4  [35, 662, 966, 526, 353, 908, 624, 720, 630, 3...    2.0    1.0   

                                        dis_emb_clip  


In [18]:
info_data2.to_parquet("data/MicroLens_1M_x1/EMB_all_emb_cb08_d.parquet")